In [34]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Initialize the T5 model and tokenizer
model_name = "geektech/flan-t5-small-gpt4-ce"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the prompt template
def generate_relevance_prompt(question, document):
    return (
            f"You are a grader assessing the relevance of a retrieved document to a user question. "
            f"User question: {question} "
            f"Retrieved document: {document} "
            f"compare the semantic meaning of both and"
            f"Rate the relevance on a scale from -1.0 to 1.0 where -1.0 means not relevant at all and 1.0 means highly relevant."
    )

# Function to check relevance and apply a threshold
def check_relevance_with_threshold(question, context, threshold=50):
    # Generate the prompt for relevance scoring
    prompt = generate_relevance_prompt(question, context)
    inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=512).to(device)

    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=10)  # Increase max_length if the response is longer

    # Decode the output and try to convert it to a score
    relevance_score_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    print(relevance_score_text)

    try:
        # Convert the text response to a numerical score
        relevance_score = float(relevance_score_text)
    except ValueError:
        # If conversion fails, assume a default score (optional)
        relevance_score = 0.0

    # Check if the score meets the threshold for relevance
    is_relevant = relevance_score >= threshold
    return relevance_score, is_relevant

# Example question and context
questions = ["Who was the producer of Escape?"]
# # contexts = ["Tommy Rettig, and Brian Keith. The announcers were Jack McCoy and Elliott Lewis. Escape (1950 TV series) Escape was a 30-minute live American dramatic anthology television series produced and directed for CBS by Wyllis Cooper. Narrated by William Conrad, the series was the television counterpart to a successful CBS Radio series of the same name (1947–54). There were a total of thirteen episodes airing on CBS from January 5, 1950 to March 30, 1950. According to \"The Complete Directory to Prime Time Network and Cable TV Shows 1946–Present\", the show's stories \"depicted people attempting to deal with danger, the supernatural"]
# # questions=["Who was the producer of Escape?"]
# contexts=["mumbai is the capital of country"]
# questions = ["What is the capital of France?"]
    # contexts = ["start coding be prepared", "Paris is the capital of France."]
contexts = [
        "Paris is the capital of France."
        ]
# Define the threshold
threshold = 0.5

# Check relevance for each question-context pair
for question, context in zip(questions, contexts):
    relevance_score, is_relevant = check_relevance_with_threshold(question, context, threshold)
    print(f"Question: '{question}'\nDocument: '{context}'\nRelevance Score: {relevance_score}\nIs Relevant: {is_relevant}\n")


1.0
Question: 'Who was the producer of Escape?'
Document: 'Paris is the capital of France.'
Relevance Score: 1.0
Is Relevant: True



In [35]:
import torch
from transformers import T5Tokenizer, T5ForSequenceClassification
from tqdm import tqdm


class RelevanceEvaluator:
    def __init__(self, model_path, device="cpu"):
        # Load the model and tokenizer
        self.device = device
        self.tokenizer, self.model = self.load_model(model_path)

    def load_model(self, model_path):
        # Initialize tokenizer and model
        tokenizer = T5Tokenizer.from_pretrained(model_path)
        model = T5ForSequenceClassification.from_pretrained(model_path, num_labels=1)
        model.to(self.device)
        model.eval()
        return tokenizer, model

    def evaluate_relevance(self, question, contexts):
        # Generate relevance scores for each context in relation to the question
        scores = []
        for context in tqdm(contexts, desc="Evaluating contexts"):
            input_text = f"{question} [SEP] {context}"
            inputs = self.tokenizer(input_text, return_tensors="pt", padding="max_length", max_length=512)
            with torch.no_grad():
                output = self.model(inputs["input_ids"].to(self.device), attention_mask=inputs["attention_mask"].to(self.device))
            # Append the relevance score for the context
            scores.append(float(output.logits.cpu()))
        print(scores)
        return scores

    def flag_relevance(self, scores, threshold1, threshold2):
        # Convert scores into relevance flags based on thresholds
        flags = []
        for score in scores:
            if score >= threshold1:
                flags.append(2)  # High relevance
            elif score >= threshold2:
                flags.append(1)  # Moderate relevance
            else:
                flags.append(0)  # Low relevance
        return flags

    def check_context_relevance(self, question, contexts, threshold1, threshold2):
        # Main method to evaluate and flag relevance for each context
        scores = self.evaluate_relevance(question, contexts)
        flags = self.flag_relevance(scores, threshold1, threshold2)
        return flags


# Example usage
if __name__ == "__main__":
    # Define model path and initialize the relevance evaluator
    model_path = "google/flan-t5-base"
    evaluator = RelevanceEvaluator(model_path, device="cpu")

    # Define input question, list of contexts, and threshold values
    question = "What is the capital of France?"
    # contexts = ["start coding be prepared", "Paris is the capital of France."]
    contexts = [
        "Escape Artists Escape Artists Productions, LLC, commonly known as Escape Artists and distinct from Escape Artists, Inc. (pod caster), is an independently financed motion picture and television production company with a first look non-exclusive deal at Sony Pictures Entertainment, headed by partners Steve Tisch, Todd Black, and Jason Blumenthal. In 2001, Todd Black and Jason Blumenthal’s Black & Blu merged with the Steve Tisch Company to form Escape Artists. The first produced movie under the Escape Artists banner was \"A Knight's Tale\", starring Heath Ledger in 2001. In the fall of 2005, Escape Artists released \"The Weather Man\", directed by",
        "Tommy Rettig, and Brian Keith. The announcers were Jack McCoy and Elliott Lewis. Escape (1950 TV series) Escape was a 30-minute live American dramatic anthology television series produced and directed for CBS by Wyllis Cooper. Narrated by William Conrad, the series was the television counterpart to a successful CBS Radio series of the same name (1947–54). There were a total of thirteen episodes airing on CBS from January 5, 1950 to March 30, 1950. According to \"The Complete Directory to Prime Time Network and Cable TV Shows 1946–Present\", the show's stories \"depicted people attempting to deal with danger, the supernatural",
        "cat chases rat so rat runs",
        "Paris is the capital of France."
        ]
    threshold1 = 0.5  # Threshold for high relevance
    threshold2 = 0.1  # Threshold for moderate relevance

    # Run the relevance evaluation
    relevance_flags = evaluator.check_context_relevance(question, contexts, threshold1, threshold2)
    print("Relevance Flags:", relevance_flags)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating contexts: 100%|██████████| 4/4 [00:18<00:00,  4.69s/it]

[-0.028868883848190308, 0.10891035199165344, 0.08477643132209778, 0.036076128482818604]
Relevance Flags: [0, 1, 0, 0]
